In [2]:
# Install external libraries
! pip install -U seqeval datasets accelerate evaluate conllu
! pip install -U transformers torch torchvision torchaudio fastai
! nvidia-smi
! pip -q install -U datasets transformers accelerate scikit-learn nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 20.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=d817773bb3bb05912dd056559e0976aa9cef5f1ac58b0baa852a97556f647637
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver doe

In [3]:
import json, random, string
from pathlib import Path
from typing import Dict, Tuple

import numpy as np
import pandas as pd

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

OUTDIR = Path("/content/nlp_pipeline_outputs")
OUTDIR.mkdir(exist_ok=True, parents=True)
print("Output dir:",
 OUTDIR)


Output dir: /content/nlp_pipeline_outputs


In [4]:
from datasets import load_dataset

def load_imdb_splits(seed: int = 42):
    ds = load_dataset("imdb")
    split = ds["train"].train_test_split(test_size=0.2, seed=seed)  # make a validation split
    return split["train"], split["test"], ds["test"]  # train, valid, test

def to_df(ds):
    return pd.DataFrame({"text": ds["text"], "label": ds["label"]})

train_ds, valid_ds, test_ds = load_imdb_splits(SEED)
df_train, df_valid, df_test = map(to_df, [train_ds, valid_ds, test_ds])

len(df_train), len(df_valid), len(df_test)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

(20000, 5000, 25000)

# NLTK preprocessing (classical baseline)

In [5]:
import nltk, string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, wordpunct_tokenize

# downloads (cached in Colab)
nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)

# Ensure resources (handles newer NLTK that expects 'punkt_tab')
for pkg in ("punkt", "punkt_tab", "stopwords", "wordnet", "omw-1.4"):
    try:
        nltk.download(pkg, quiet=True)
    except Exception:
        pass  # ignore if not available

STOP = set(stopwords.words("english"))
LEM  = WordNetLemmatizer()
PUNCT = set(string.punctuation)

def _tokenize_safe(text: str):
    try:
        return word_tokenize(text)
    except LookupError:
        # Fallback if punkt/punkt_tab is missing or download hiccups
        return wordpunct_tokenize(text)

def clean_text(doc: str) -> str:
    doc = doc.lower()
    doc = "".join(ch for ch in doc if ch not in PUNCT)
    toks = _tokenize_safe(doc)
    toks = [LEM.lemmatize(t) for t in toks if t not in STOP and len(t) > 2]
    return " ".join(toks)

for df in (df_train, df_valid, df_test):
    df["clean"] = df["text"].apply(clean_text)

X_train, y_train = df_train["clean"].to_numpy(), df_train["label"].to_numpy()
X_valid, y_valid = df_valid["clean"].to_numpy(), df_valid["label"].to_numpy()
X_test,  y_test  = df_test["clean"].to_numpy(),  df_test["label"].to_numpy()

print("Sample cleaned text:", X_train[0][:200])


Sample cleaned text: stage adaptation often major fault often come looking like film camera simply placed stage night mother sidney lumets direction keep film alive especially difficult since picture offered real challeng


# TF-IDF + Logistic Regression

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def eval_metrics(y_true, y_pred, tag=""):
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    print(f"[{tag}] acc={acc:.4f}  prec={prec:.4f}  rec={rec:.4f}  f1={f1:.4f}")
    return {"accuracy":acc, "precision":prec, "recall":rec, "f1":f1}

tfidf_logreg = Pipeline([
    ("tfidf", TfidfVectorizer(min_df=2, ngram_range=(1,2), max_features=80_000)),
    ("lr", LogisticRegression(max_iter=2000, n_jobs=-1))
])

tfidf_logreg.fit(X_train, y_train)
val_pred = tfidf_logreg.predict(X_valid)
tst_pred = tfidf_logreg.predict(X_test)

val_cls = eval_metrics(y_valid, val_pred, "TFIDF+LR / VALID")
tst_cls = eval_metrics(y_test,  tst_pred, "TFIDF+LR / TEST")

# Simple interpretability: top weighted terms
vec = tfidf_logreg.named_steps["tfidf"]
lr  = tfidf_logreg.named_steps["lr"]
feats = np.array(vec.get_feature_names_out())
coefs = lr.coef_[0]
top_pos = feats[np.argsort(coefs)][-20:][::-1]
top_neg = feats[np.argsort(coefs)][:20]

print("\n[TFIDF+LR] Top positive terms:", ", ".join(top_pos))
print("[TFIDF+LR] Top negative terms:", ", ".join(top_neg))


[TFIDF+LR / VALID] acc=0.8904  prec=0.8867  rec=0.8958  f1=0.8912
[TFIDF+LR / TEST] acc=0.8840  prec=0.8819  rec=0.8866  f1=0.8843

[TFIDF+LR] Top positive terms: great, excellent, perfect, wonderful, best, favorite, loved, amazing, love, well, today, fun, brilliant, one best, life, also, enjoyed, still, beautiful, highly
[TFIDF+LR] Top negative terms: worst, bad, awful, waste, boring, poor, nothing, terrible, worse, horrible, unfortunately, script, dull, supposed, stupid, even, minute, poorly, annoying, lack


# DistilBERT fine-tuning

In [7]:
import torch, evaluate
print("Torch CUDA available?", torch.cuda.is_available())

from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer,
    EarlyStoppingCallback, set_seed
)
set_seed(SEED)

MODEL = "distilbert-base-uncased"
tok = AutoTokenizer.from_pretrained(MODEL)

def tok_fn(batch):
    return tok(batch["text"], truncation=True)

tok_train = train_ds.map(tok_fn, batched=True, remove_columns=["text"])
tok_valid = valid_ds.map(tok_fn, batched=True, remove_columns=["text"])
tok_test  = test_ds.map(tok_fn,  batched=True, remove_columns=["text"])

collator = DataCollatorWithPadding(tokenizer=tok)
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=2)

# metrics
acc_metric = evaluate.load("accuracy")
prec_metric = evaluate.load("precision")
rec_metric  = evaluate.load("recall")
f1_metric   = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":  acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec_metric.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall":    rec_metric.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1":        f1_metric.compute(predictions=preds, references=labels, average="binary")["f1"],
    }

# FP16 on GPU for speed
use_fp16 = torch.cuda.is_available()

args = TrainingArguments(
    output_dir="/content/distilbert_imdb",
    do_eval=True,
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,        # save every N steps
    seed=42,
    fp16=True,             # if GPU
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tok_train,
    eval_dataset=tok_valid,
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

print("\n=== DistilBERT training (GPU) ===")
trainer.train()
valid_bert = trainer.evaluate(tok_valid)
print("[BERT / VALID]", valid_bert)

test_pred = trainer.predict(tok_test)
bert_test = compute_metrics((test_pred.predictions, test_pred.label_ids))
print("[BERT / TEST]", bert_test)


Torch CUDA available? True


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nicolam8 (nicolam8-la-trobe-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.553400
100,0.338800
150,0.283600
200,0.268000
250,0.263700
300,0.272200
350,0.235800
400,0.239500
450,0.233000
500,0.200600


[BERT / VALID] {'eval_loss': 0.21005678176879883, 'eval_accuracy': 0.9154, 'eval_precision': 0.9147749900438072, 'eval_recall': 0.9166001596169194, 'eval_f1': 0.9156866653378513, 'eval_runtime': 19.2222, 'eval_samples_per_second': 260.116, 'eval_steps_per_second': 8.168, 'epoch': 1.0}
[BERT / TEST] {'accuracy': 0.91996, 'precision': 0.9125343811394891, 'recall': 0.92896, 'f1': 0.9206739345887017}


In [8]:
report = {
    "tfidf_logreg": {
        "valid": val_cls,
        "test":  tst_cls,
        "top_positive_terms": list(top_pos),
        "top_negative_terms": list(top_neg),
    },
    "distilbert": {
        "valid": {k: float(v) for k, v in valid_bert.items() if k.startswith("eval_")},
        "test":  {k: float(v) for k, v in bert_test.items()},
    },
    "notes": "Classical uses NLTK-cleaned text + TF-IDF; DistilBERT uses raw-tokenized text.",
}

out_path = OUTDIR / "summary_report.json"
with open(out_path, "w") as f:
    json.dump(report, f, indent=2)

out_path, report["distilbert"]["test"], report["tfidf_logreg"]["test"]


(PosixPath('/content/nlp_pipeline_outputs/summary_report.json'),
 {'accuracy': 0.91996,
  'precision': 0.9125343811394891,
  'recall': 0.92896,
  'f1': 0.9206739345887017},
 {'accuracy': 0.88396,
  'precision': 0.8819129466061908,
  'recall': 0.88664,
  'f1': 0.8842701559819683})